In [1]:
import numpy as np
import sympy as sp
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [2]:
t,g,m1,m2,m3,l1,l2,l3 = sp.symbols(r't g m_1 m_2 m_3 l_1 l_2 l_3')

In [3]:
theta1, theta2, theta3 = sp.Function(r'theta_1')(t), sp.Function(r'theta_2')(t), sp.Function(r'theta_3')(t)

In [4]:
theta1d = sp.diff(theta1,t)
theta2d = sp.diff(theta2,t)
theta3d = sp.diff(theta3,t)

theta1dd = sp.diff(theta1d,t)
theta2dd = sp.diff(theta2d,t)
theta3dd = sp.diff(theta3d,t)

In [5]:
x1 = l1*sp.sin(theta1)
y1 = -l1*sp.cos(theta1)

x2 = x1 + l2*sp.sin(theta2)
y2 = y1 - l2*sp.cos(theta2)

x3 = x2 + l3*sp.sin(theta3)
y3 = y2 - l3*sp.cos(theta3)

In [6]:
x1_f = sp.lambdify((theta1,theta2,theta3,l1,l2,l3), x1)
y1_f = sp.lambdify((theta1,theta2,theta3,l1,l2,l3), y1)

x2_f = sp.lambdify((theta1,theta2,theta3,l1,l2,l3), x2)
y2_f = sp.lambdify((theta1,theta2,theta3,l1,l2,l3), y2)

x3_f = sp.lambdify((theta1,theta2,theta3,l1,l2,l3), x3)
y3_f = sp.lambdify((theta1,theta2,theta3,l1,l2,l3), y3)

In [7]:
# Kinetic Energy
T = (1/2)*m1*(sp.diff(x1,t)**2 + sp.diff(y1,t)**2) + (1/2)*m2*(sp.diff(x2,t)**2+sp.diff(y2,t)**2) + (1/2)*m3*(sp.diff(x3,t)**2+sp.diff(y3,t)**2)
# Potential Energy
V = m1*g*y1 + m2*g*y2 + m3*g*y3
# Lagrangian
L = T - V

In [8]:
Leq1 = sp.diff(sp.diff(L,theta1d),t) - sp.diff(L,theta1)
Leq2 = sp.diff(sp.diff(L,theta2d),t) - sp.diff(L,theta2)
Leq3 = sp.diff(sp.diff(L,theta3d),t) - sp.diff(L,theta3)

In [9]:
sp.solve(Leq3,theta3dd)

[(-g*sin(theta_3(t)) + l_1*sin(theta_1(t) - theta_3(t))*Derivative(theta_1(t), t)**2 - l_1*cos(theta_1(t) - theta_3(t))*Derivative(theta_1(t), (t, 2)) + l_2*sin(theta_2(t) - theta_3(t))*Derivative(theta_2(t), t)**2 - l_2*cos(theta_2(t) - theta_3(t))*Derivative(theta_2(t), (t, 2)))/l_3]

In [ ]:
sol = sp.solve([Leq1,Leq2,Leq3],(theta1dd, theta2dd,theta3dd))

let,
* $o = \frac{d\theta_1}{dt} \implies \frac{do}{dt} = \frac{d^2\theta_1}{dt^2}$
* $\omega = \frac{d\theta_2}{dt} \implies \frac{d\omega}{dt} = \frac{d^2\theta_2}{dt^2}$

In [ ]:
theta1dd_f = sp.lambdify((theta1, theta2, theta3, theta1d, theta2d, theta3d l1, l2,l3, m1, m2, m3, g), sol[theta1dd])
theta2dd_f = sp.lambdify((theta1, theta2, theta3, theta1d, theta2d, theta3d l1, l2,l3, m1, m2, m3, g), sol[theta2dd])
theta3dd_f = sp.lambdify((theta1, theta2, theta3, theta1d, theta2d, theta3d l1, l2,l3, m1, m2, m3, g), sol[theta3dd])

In [ ]:
def dSdt(S,t,l1,l2,m1,m2,g):
    theta1, theta2, o, w, zi = S
    return [
        o,
        w,
        zi,
        theta1dd_f(theta1, theta2, theta3, o, w, zi, l1, l2,l3 , m1, m2, m3, g),
        theta2dd_f(theta1, theta2, theta3, o, w, zi, l1, l2,l3 , m1, m2, m3, g),
        theta3dd_f(theta1, theta2, theta3, o, w, zi, l1, l2,l3 , m1, m2, m3, g)
    ]

In [ ]:
l1 = 2
l2 = 2
l3 = 2
m1 = 3
m2 = 2
m3 = 2
g = 9.8
S0 = [2,1,1,1,1,1]
t = np.linspace(0,40,1001)
ans = odeint(dSdt,y0=S0 ,t=t, args=(l1,l2,l3,m1,m2,m3,g))

In [ ]:
ans.shape

In [ ]:
plt.plot(t, ans[:, 0], label=r'$\theta_1$')
plt.plot(t, ans[:, 1], label=r'$\theta_2$')
plt.xlabel('Time [s]')
plt.ylabel('Angle [rad]')
plt.legend()
plt.show()

In [ ]:
def get_pos():
    x_1 = x1_f(ans[:,0],ans[:,1],ans[:,2],l1,l2,l3)
    y_1 = y1_f(ans[:,0],ans[:,1],ans[:,2],l1,l2,l3)
    
    x_2 = x2_f(ans[:,0],ans[:,1],ans[:,2],l1,l2,l3)
    y_2 = y2_f(ans[:,0],ans[:,1],ans[:,2],l1,l2,l3)
    
    x_3 = x3_f(ans[:,0],ans[:,1],ans[:,2],l1,l2,l3)
    y_3 = y3_f(ans[:,0],ans[:,1],ans[:,2],l1,l2,l3)
    
    return np.array([x_1 , y_1, x_2, y_2, x_3, y_3])

In [ ]:
get_pos()[0]

In [ ]:
np.save('D:\dataset\triple_pendulum.npy',get_pos())

In [ ]:
x = np.linspace(0,2*np.pi,10000)